In [18]:
import numpy as np
import numpy as np
from sklearn.metrics import jaccard_score
from torchmetrics import JaccardIndex, Dice
from torch import tensor
import torch.nn as nn
import torch
import torch.utils
import torch.utils.data
from sklearn.model_selection import train_test_split
import attention as at
from torch.functional import F
from torchvision.ops.focal_loss import sigmoid_focal_loss
from skimage import io

In [2]:
a = np.random.randint(0,2,(6,194,194))
b = np.random.randint(0,2,(6,194,194))

In [3]:
intersection = np.logical_and(a,b)

In [4]:
unioun = np.logical_or(a,b)

In [5]:
print(intersection.sum()/unioun.sum())

0.3345054011480758


In [21]:
def jaccard_index(mask1, mask2):
    intersection = np.logical_and(mask1, mask2) + 1e-10
    union = np.logical_or(mask1, mask2) + 1e-10
    # intersection = mask1 * mask2
    # union = mask1 + mask2
    iou = (intersection.sum() / union.sum())
    return iou

def dice_loss(pred, target, smooth=1):
    pred = pred.contiguous()
    target = target.contiguous()

    intersection = (pred * target).sum(dim=0).sum(dim=0)

    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=0).sum(dim=0) + target.sum(dim=0).sum(dim=0) + smooth)))

    return loss.mean()

# Example usage:
# Assuming mask1 and mask2 are binary masks represented as NumPy arrays
mask1 = np.array([[0, 0, 0],
                  [1, 1, 1],
                  [0, 0, 1]])

mask2 = np.array([[0, 1, 1],
                  [1, 1, 0],
                  [0, 0, 1]])

iou = jaccard_index(mask1, mask2)
print("Jaccard index calculated :", iou)
print('Jaccard index from sklearn :', jaccard_score(mask1, mask2, average='micro', zero_division=1))
print('Jaccard index from torchmetrics :', JaccardIndex(task='binary')(tensor(mask1), tensor(mask2)).item())
print('Dice from torchmetrics :', 1 - Dice(multiclass=False,num_classes=1)(tensor(mask1), tensor(mask2)))
print('Dice calculated :', dice_loss(tensor(mask1), tensor(mask2)).item())

Jaccard index calculated : 0.500000000075
Jaccard index from sklearn : 0.5
Jaccard index from torchmetrics : 0.5
Dice from torchmetrics : tensor(0.3333)
Dice calculated : 0.30000001192092896


In [5]:
np.zeros((2,3,3))[0].shape

(3, 3)

In [140]:
print('Binary Cross Entropy :', nn.BCELoss(weight=tensor(1))(tensor(mask1,dtype=torch.float32), tensor(mask2,dtype=torch.float32)).item())

Binary Cross Entropy : 22.22222137451172


In [128]:
target = io.imread('target.png') // 255
predict = np.zeros_like(target) // 255
# predict = io.imread('predict.png',as_gray=True) // 255

In [130]:
iou = jaccard_index(target, predict)
print("Jaccard index calculated :", iou)
print('Jaccard index from sklearn :', jaccard_score(target.flatten(), predict.flatten(), average='binary'))
print('Jaccard index from torchmetrics :', JaccardIndex(task='binary')(tensor(target), tensor(predict)).item())
print('Dice from torchmetrics :', 1 - Dice(multiclass=False,num_classes=1)(tensor(target), tensor(predict)))
print('Dice calculated :', dice_loss(tensor(target), tensor(predict)).item())

Jaccard index calculated : 0.0
Jaccard index from sklearn : 0.0
Jaccard index from torchmetrics : 0.0
Dice from torchmetrics : tensor(1.)
Dice calculated : 0.9919354915618896


In [144]:
print('Binary Cross Entropy :', nn.BCELoss(weight=tensor(10))(tensor(predict,dtype=torch.float32), tensor(target,dtype=torch.float32)).item())

Binary Cross Entropy : 1.8768310546875


In [161]:
loss = sigmoid_focal_loss(tensor(predict,dtype=torch.float32), tensor(target,dtype=torch.float32), gamma=0.1, alpha=0, reduction='mean')
print('Focal Loss :', loss)

Focal Loss : tensor(0.6455)


In [35]:
tensor(tensor(mask1))

C:\Users\acer\AppData\Local\Temp\ipykernel_3048\2214675989.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor(tensor(mask1))


tensor([[0, 1, 0],
        [1, 1, 0],
        [0, 0, 1]], dtype=torch.int32)

In [10]:
loss = 1 - Dice(multiclass=False).req(tensor(mask1), tensor(mask2))
loss.requires_grad = True
loss.backward()
print(loss.grad)

tensor(1.)


In [25]:
criterion_dice = Dice(multiclass=False,num_classes=1)
criterion_dice.requires_grad_(True)
loss = criterion_dice(tensor(mask1), tensor(mask2))
loss = loss.int()
loss.requires_grad = True
loss.backward()
print(loss.grad)

RuntimeError: only Tensors of floating point and complex dtype can require gradients

In [10]:
softmax = nn.Softmax(dim=1)
input = (torch.randn((3,3)) > 0) * 1
print(input.reshape((-1,3,3)))
print(input.max().item())
print(input.min().item())
# output = softmax(input)
# print(output)
sigmoid = nn.Sigmoid()
output = sigmoid(input)
print(output)

tensor([[[1, 0, 0],
         [1, 1, 1],
         [1, 0, 1]]])
1
0
tensor([[0.7311, 0.5000, 0.5000],
        [0.7311, 0.7311, 0.7311],
        [0.7311, 0.5000, 0.7311]])


In [51]:
train_1, test_1 = torch.utils.data.random_split(range(10), [0.7,0.3])
train_2, test_2 = train_test_split(range(10), test_size=0.3, random_state=42)
train_3, test_3 = train_test_split(range(10), test_size=0.3, random_state=42)

In [52]:
print(train_2,test_2)
print(train_3,test_3)

[0, 7, 2, 9, 4, 3, 6] [8, 1, 5]
[0, 7, 2, 9, 4, 3, 6] [8, 1, 5]


In [6]:
arr = np.random.randint(0, 100, (10, 12))
print(arr)

[[55 24 20 48 18 10 44 33  2 24 54 32]
 [89 42 73 71 69  8 74  7 75  0 82 45]
 [63 72 40 69  0 69 57 59 93 61 75 31]
 [91 72 41 90 85 81 41 96 54 22 50 67]
 [91 58 33 55 89 81 47 86 78 55 10 71]
 [85 21 83 23 72 81 78 42 69 66 64 31]
 [56 28 74 11 14 52 85 82 65 85 28 89]
 [ 5 29 80 44  1 74 50 36 49 72 26  5]
 [75 62  6 36  1 11 82 36 36 15 94 73]
 [29 35 15  4 94 27 81 57 20 95 59 54]]


In [ ]:
np.transpose(arr,)

In [7]:
arr.repeat(2, axis=1).transpose([0, 2, 3, 1]).astype(np.uint8)

ValueError: axes don't match array